In [1]:
import requests                     # -> 웹 페이지 요청 모듈
import urllib                       # -> URLEncoding을 위한 모듈
from bs4 import BeautifulSoup       # -> 웹 페이지 소스코드 분석 모듈
from pandas import DataFrame        # -> 데이터 프레임

In [2]:
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.120 Safari/537.36"
header_info = {'User-agent': user_agent, 'referer': None}
header_info

{'User-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.120 Safari/537.36',
 'referer': None}

In [3]:
keyword = "팬츠"   # 검색어

In [4]:
base_url = "http://itproject.ezenac.co.kr/springmyshop/product/best"

In [5]:
base_param = {"page" : 1, "keyword" : keyword}
base_param

{'page': 1, 'keyword': '팬츠'}

In [20]:
# 수집 결과를 누적할 빈 리스트
data_list = []

# 페이지 번호 초기화
base_param["page"] = 1

# 직접 접속하기 전까지는 검색결과가 몇 페이지인지 모르는 상태
while True:
    #------------------------------------------
    # 1) 접속할 주소 준비하기
    #------------------------------------------
    # URLEncoding
    query = urllib.parse.urlencode(base_param)

    # 접속할 주소 확정
    content_url = base_url + "?" + query
    print(content_url)
    #break
    
    
    #------------------------------------------
    # 2) 페이지에 접속하기
    #------------------------------------------
    r = requests.get(content_url, headers=header_info)
    print(r)

    # 결과 검사
    if r.status_code != 200:
        # 에러코드와 에러메시지를 문자열로 구성
        err_msg = "%d %s 에러가 발생했습니다." % (r.status_code, r.reason)
        # 무한루프 처리 중이므로 에러 발생시 continue를 명시하면 반복 종료가 되지 않는다.
        # 그러므로 강제로 에러를 발생시켜 반복 수행을 종료해야 한다.
        raise Exception(err_msg)

    r.encoding = "utf-8"  # 인코딩 설정

    # 웹 페이지의 소스코드 HTML 분석 객체로 생성
    soup = BeautifulSoup(r.text, 'html.parser')
    #print(soup)
    #break
        
    #------------------------------------------
    # 3) 상품정보 추출
    #------------------------------------------
    product_item = soup.select(".product-item")
    
    # 결과 검사
    if len(product_item) == 0:
        # 에러코드와 에러메시지를 문자열로 구성
        err_msg = "지정된 선택자에 대해서 인식된 구조가 없습니다."
        # 강제로 에러를 발생시킨다.
        raise Exception(err_msg)
    
    print("가져온 상품 수: %d" % len(product_item))
    #print(product_item)
    #break
    
    # 상품 수 만큼 반복
    for item in product_item:
        # 상품명
        name = item.select("h3")
        name = name[0].text.strip()
        print(name)
        
        # 상품가격
        price = item.select(".price")
        price_str = price[0].text.strip()
        
        # 정가와 할인가가 나누어져 있는 경우
        if price_str.count("원") == 2:
            origin_price = price[0].select(".cancel-price")[0].text.strip()
            sale_price = price[0].select(".sale-price")[0].text.strip()
        # 정가만 있는 경우
        else:
            origin_price = price_str
            sale_price = None
        
        # 정가 문자열에서 불필요한 글자(=`원`,',')와 공백을 제거하고 정수로 변환
        origin_price = int(origin_price.replace("원", "").replace(",", "").strip())
        
        # 할인가는 없는 경우도 있으므로 값이 존재하는 경우만 처리
        if sale_price:
            sale_price = int(sale_price.replace("원", "").replace(",", "").strip())
    
        # 상품설명
        desc = item.select(".desc")[0].text.strip()
        
        product_dic = {"상품명": name, 
                       "정가": origin_price, 
                       "할인가": sale_price, 
                       "설명": desc}
        
        # 하나의 상품 정보를 리스트에 병합
        data_list.append(product_dic)
        print(product_dic)
    
    #------------------------------------------
    # 4) 전체 페이지수를 가져와서 반복 실행 여부 판단하기
    #------------------------------------------
    text_muted = soup.select(".text-muted")
    # -> (1/7Pages.)
    page_info = text_muted[0].text.strip()
    p = page_info.find("/")
    max_page = int(page_info[p+1:-7])
    
    # 원하는 조건이 성립한 경우 처리 중단
    if base_param["page"] >= max_page:
        break
        
    # 페이지 번호 1 증가
    base_param["page"] += 1

http://itproject.ezenac.co.kr/springmyshop/product/best?page=1&keyword=%ED%8C%AC%EC%B8%A0
<Response [200]>
가져온 상품 수: 36
블린체휘트니스팬츠
{'상품명': '블린체휘트니스팬츠', '정가': 25900, '할인가': None, '설명': '★휘트니스룩★텐션좋은 쫀쫀한 스판기에 기능성원단으로 부담없이 입기 좋은 휘트니스 팬츠입니다~  size(S~L)'}
플러튼휘트니스팬츠
{'상품명': '플러튼휘트니스팬츠', '정가': 25900, '할인가': None, '설명': '다리 라인을 매끄럽게 정리해주는 쫀쫀한 신축성에 아랫배까지 잡아주는 긴 밑위기장으로 핏이 예쁜 피트니스팬츠입니다~ size(S,M)'}
메이건7부트레이닝팬츠
{'상품명': '메이건7부트레이닝팬츠', '정가': 15900, '할인가': None, '설명': '★휘트니스룩★ 곡선 옆라인이 바디라인을 더욱 날씬하게! 쫀쫀한 허리밴딩이 아랫배를 꽉 잡아준답니다 !  free size (44~66)'}
루더스휘트니스치마팬츠
{'상품명': '루더스휘트니스치마팬츠', '정가': 19900, '할인가': None, '설명': '스커트가 레이어드된 피트니스팬츠입니다. 배색라인 포인트와 쫀쫀한 텐션으로 다리라인을 정리해줘요~ free size(44~66)'}
론사선트레이닝팬츠
{'상품명': '론사선트레이닝팬츠', '정가': 21900, '할인가': 13140, '설명': '★휘트니스룩★옆라인 사선 배색 포인트로 더욱 얇아보이는 다리라인으로 만들어준답니당 ~  free size (44~통통66)'}
페버휘트니스팬츠
{'상품명': '페버휘트니스팬츠', '정가': 19900, '할인가': None, '설명': '★휘트니스룩★ 베이직한 디자인으로 데일리, 스포티하게 즐기기 좋은 휘트니스 팬츠! size(S~L)'}
치크네온트레이닝팬츠
{'상품명': '치크네온트레이닝팬츠', '정가': 17900, '할인가': None, '설명': '★휘

In [21]:
df = DataFrame(data_list)
df

,상품명,정가,할인가,설명
0,블린체휘트니스팬츠,25900,NaN,★휘트니스룩★텐션좋은 쫀쫀한 스판기에 기능성원단으로 부담없이 입기 좋은 휘트니스 팬...
1,플러튼휘트니스팬츠,25900,NaN,다리 라인을 매끄럽게 정리해주는 쫀쫀한 신축성에 아랫배까지 잡아주는 긴 밑위기장으로...
2,메이건7부트레이닝팬츠,15900,NaN,★휘트니스룩★ 곡선 옆라인이 바디라인을 더욱 날씬하게! 쫀쫀한 허리밴딩이 아랫배를 ...
3,루더스휘트니스치마팬츠,19900,NaN,스커트가 레이어드된 피트니스팬츠입니다. 배색라인 포인트와 쫀쫀한 텐션으로 다리라인을...
4,론사선트레이닝팬츠,21900,13140.0,★휘트니스룩★옆라인 사선 배색 포인트로 더욱 얇아보이는 다리라인으로 만들어준답니당 ...
...,...,...,...,...
240,헬로디밴딩팬츠,9900,8910.0,신축성 좋은 허리밴딩으로 편안하게~ 9가지 다양한 컬러로 넓은 초이스~ one si...
241,코븐 4부데님팬츠,16800,NaN,은은한 워싱디자인과 밑단 컷팅스타일로 빈티지하게! size(S~XXL)
242,[인싸팬츠12탄]레트로 플라워 패턴팬츠,36800,33120.0,찰랑찰랑 시원한 느낌의 패턴 소재로 편안한 밴딩 팬츠 아이템! FREE(44~66)
243,러뷰드 린넨와이드팬츠,49800,44820.0,★자체제작 + 루즈핏★ 허리는밴딩으로 편안하게~ 입어주면 우아한 룩으로 완성시켜주는...


In [22]:
df.to_excel('상품정보(%s).xlsx' %keyword, index=False)